In [1]:
import numpy as np 
import pandas as pd
import os
import re
#%tensorflow_version 2.x 
import tensorflow as tf
from tensorflow.python.keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Embedding, GRU, LSTM, Dropout, Bidirectional, Flatten
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.models import Sequential


Using TensorFlow backend.


In [2]:
data = pd.read_csv('train.csv')

In [3]:
import re
def cleaner(old_text):
    cleaned_text = [] 
    for sentences in old_text :
        hold = re.sub('[^a-zA-Z]',' ', sentences)
        hold = hold.lower()
        cleaned_text.append(hold)
    return cleaned_text

In [4]:
data['cleaned_text'] = cleaner(data.text)
data.head()

,id,keyword,location,text,target,cleaned_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake m...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation or...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as ...


In [5]:
x = data['cleaned_text']
y = data['target'].values

In [6]:
length = len(x)
length

7613

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
sequence = tokenizer.texts_to_sequences(x)

In [8]:
word_index = tokenizer.word_index
len(word_index)

22127

In [9]:
padded = pad_sequences(sequence, maxlen= 15, padding='post')
padded[0]

array([ 130, 4742,   27,    4,  851,    9,   22,  269,  152, 1597, 4743,
         89,   43,    0,    0])

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(padded, y, test_size = 0.20, random_state = 42)

In [11]:
len(x_train)/2

3045.0

In [12]:
model = Sequential()

model.add(Embedding(22128, 150, input_length= 15))
model.add(GRU(32,return_sequences=True, dropout=0.3, recurrent_dropout=0.2))

model.add(Dense(output_dim = 24,init = 'uniform', activation = 'relu'))
model.add(Dropout(p = 0.3))

model.add(Flatten())
model.add(Dense(output_dim = 1,init = 'uniform', activation = 'sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

#https://keras.io/layers/core/
#https://keras.io/layers/recurrent/
#https://keras.rstudio.com/reference/fit.html

W1230 02:47:22.583335  5060 deprecation_wrapper.py:119] From C:\Users\Kenechi\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1230 02:47:22.647379  5060 deprecation_wrapper.py:119] From C:\Users\Kenechi\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1230 02:47:22.655384  5060 deprecation_wrapper.py:119] From C:\Users\Kenechi\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1230 02:47:22.845515  5060 deprecation_wrapper.py:119] From C:\Users\Kenechi\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1230 02:47:22.868525  5060 deprecatio

In [13]:
x_train_val = x_train[:3045]
x_train_par = x_train[3045:]

y_train_val = y_train[:3045]
y_train_par = y_train[3045:]

In [14]:
model.fit(x_train_par, y_train_par, batch_size=10, epochs=3, validation_data = (x_train_val,y_train_val), verbose =1)

Train on 3045 samples, validate on 3045 samples
Epoch 1/3
3045/3045 [==============================] - 22s 7ms/step - loss: 0.6042 - acc: 0.6588 - val_loss: 0.5082 - val_acc: 0.7527
Epoch 2/3
3045/3045 [==============================] - 17s 6ms/step - loss: 0.4244 - acc: 0.8167 - val_loss: 0.4879 - val_acc: 0.7819
Epoch 3/3
3045/3045 [==============================] - 17s 6ms/step - loss: 0.3298 - acc: 0.8693 - val_loss: 0.5057 - val_acc: 0.7849


In [15]:
y_pred= model.predict(x_test)

In [16]:
from sklearn.metrics import confusion_matrix, classification_report
check = classification_report(y_test, y_pred.round())
print(check)

             precision    recall  f1-score   support

          0       0.78      0.86      0.82       874
          1       0.78      0.67      0.72       649

avg / total       0.78      0.78      0.77      1523



In [17]:
test = pd.read_csv('test.csv')
test['cleaned_text'] = cleaner(test.text)
X_t = tokenizer.texts_to_sequences(test.cleaned_text)

In [18]:
padded_xt = pad_sequences(X_t, maxlen= 15, padding='post')

In [19]:
submit2 = pd.read_csv('sample_submission.csv')
submit2.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [20]:
y_pred2 = model.predict(padded_xt)

In [21]:
submit2['target'] =y_pred2
submit2['target'] = submit2['target'].apply(lambda x : 0 if x<=0.5 else 1)
submit2.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [22]:
#submit2.to_csv("submission3.csv", index=False)